# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [22]:
from IPython.display import HTML
from plotly.io import to_html

In [23]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)


root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: string (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: string (nullable = true)
 |-- EXPIRED: string (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: string (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [32]:
import plotly.express as px

# Explicitly filter dataset: remove null or zero salaries
filtered_df = df.filter((df["SALARY"].isNotNull()) & (df["SALARY"] > 0))

# Explicitly collect data directly to pandas DataFrame (bypassing distutils issue)
pdf = pd.DataFrame(
    filtered_df.select("EMPLOYMENT_TYPE_NAME", "SALARY").collect(),
    columns=["EMPLOYMENT_TYPE_NAME", "SALARY"]
)

# Create clear box plot visualizing salary distribution by employment type
fig = px.box(
    pdf,
    x="EMPLOYMENT_TYPE_NAME",
    y="SALARY",
    title="Salary Distribution by Employment Type",
    color="EMPLOYMENT_TYPE_NAME",
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Explicitly enhance visualization readability
fig.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title="Employment Type",
    yaxis_title="Salary",
    legend_title="Employment Type",
    boxmode="group"
)

HTML(to_html(fig, include_plotlyjs='cdn'))

The box plot clearly illustrates that full-time positions (>32 hours) generally offer higher median salaries compared to part-time positions. Additionally, part-time/full-time roles exhibit greater salary variability and have more extreme high-value outliers compared to purely part-time roles.

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [25]:
# Filter dataset explicitly (salary > 0)
filtered_df = df.filter(df["SALARY_FROM"] > 0)

# Aggregate explicitly: select NAICS industry and salary columns, and convert to pandas dataframe
pdf = pd.DataFrame(
    filtered_df.select("NAICS2_NAME", "SALARY_FROM").collect(),
    columns=["NAICS2_NAME", "SALARY_FROM"]
)

# Create clear box plot comparing salary variations across industries
fig = px.box(
    pdf,
    x="NAICS2_NAME",
    y="SALARY_FROM",
    title="Salary Variations Across Industries",
    color="NAICS2_NAME",
    color_discrete_sequence=px.colors.qualitative.Pastel
)

# Explicitly customize graph styles for clarity and aesthetics
fig.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title="Industry (NAICS2)",
    yaxis_title="Salary",
    legend_title="Industry",
    boxmode="group"
)

HTML(to_html(fig, include_plotlyjs='cdn'))


The box plot shows that salary distributions vary significantly across industries, with sectors like Unclassified Industry, Information, and Waste Management and Remediation Services offering higher and more dispersed salaries. In contrast, industries like Art, Entertatainment or Agriculture generally show lower and more compact salary ranges.

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [26]:
from pyspark.sql.functions import to_date, col, count, date_format
import pandas as pd
import plotly.express as px

df_with_date = df.withColumn("POSTED_DATE", to_date(col("POSTED"), "M/d/yyyy"))

# Group by week or month if all dates are too similar
df_grouped = df_with_date.withColumn("POSTED_MONTH", date_format("POSTED_DATE", "yyyy-MM")) \
                         .groupBy("POSTED_MONTH") \
                         .agg(count("*").alias("NUM_POSTINGS"))

# Convert to pandas for plotting
pdf = pd.DataFrame(df_grouped.sort("POSTED_MONTH").collect(), columns=["POSTED_MONTH", "NUM_POSTINGS"])

# Plot line chart
fig = px.line(
    pdf,
    x="POSTED_MONTH",
    y="NUM_POSTINGS",
    title="Job Postings Over Time (Monthly)",
    markers=True
)

fig.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title="Month",
    yaxis_title="Number of Job Postings",
    plot_bgcolor="#f8f9fa"
)

HTML(to_html(fig, include_plotlyjs='cdn'))



The chart shows that the number of job postings decreased during the summer, while postings remained consistent at the beginning and end of the year. With more than 11,000 job postings each month, it reflects a high and steady demand for workers in the U.S.

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [27]:
# Group by job title and count occurrences
job_counts = df.groupBy("TITLE_NAME") \
               .agg(count("*").alias("JOB_COUNT"))

# Select Top 10 job titles by count
top_10_jobs = job_counts.orderBy(col("JOB_COUNT").desc()).limit(10)

# Convert to Pandas for Plotly
pdf = pd.DataFrame(
    top_10_jobs.collect(),
    columns=["TITLE_NAME", "JOB_COUNT"]
)

# Plot bar chart
fig = px.bar(
    pdf,
    x="TITLE_NAME",
    y="JOB_COUNT",
    title="Top 10 Most Frequently Posted Job Titles",
    color="TITLE_NAME",
    color_discrete_sequence=px.colors.qualitative.Vivid
)

# Customize styles
fig.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title="Job Title",
    yaxis_title="Number of Postings",
    xaxis_tickangle=-45
)

HTML(to_html(fig, include_plotlyjs='cdn'))


The chart shows that Data Analyst is the most frequently posted job title, with more than double the postings compared to other roles—highlighting the strong demand for data professionals. In contrast, Data Quality Analyst appears the least among the top 10, suggesting it is a more specialized or less commonly advertised role.

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [28]:
# Group by REMOTE_TYPE_NAME and count job postings
remote_counts = df.groupBy("REMOTE_TYPE_NAME") \
                  .agg(count("*").alias("JOB_COUNT"))

# Convert to Pandas for visualization
pdf = pd.DataFrame(
    remote_counts.collect(),
    columns=["REMOTE_TYPE_NAME", "JOB_COUNT"]
)

# Create pie chart
fig = px.pie(
    pdf,
    names="REMOTE_TYPE_NAME",
    values="JOB_COUNT",
    title="Proportion of Remote vs On-site Job Postings",
    color_discrete_sequence=px.colors.sequential.RdBu
)

# Customize styles
fig.update_layout(
    font_family="Arial",
    title_font_size=18,
    legend_title="Remote Type"
)

HTML(to_html(fig, include_plotlyjs='cdn'))


The pie chart reveals that a majority of job postings (78%) are for on-site positions, while remote roles make up 17.2%, and hybrid opportunities account for only 1.55%. This suggests that despite the growth of remote work trends, many employers still prioritize in-person collaboration—possibly due to the nature of the industry, security requirements, or company culture.

# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [29]:
from pyspark.sql.functions import split, explode, col, count
import pandas as pd
import plotly.express as px

# Extract and explode skills from a comma-separated list
skills_df = df.withColumn("SKILLS_NAME", explode(split(col("SKILLS_NAME"), ",\s*")))

# Group by industry and skill, then count how often each appears
industry_skills = skills_df.groupBy("NAICS2_NAME", "SKILLS_NAME") \
                           .agg(count("*").alias("SKILL_COUNT"))

# Limit to most common skills across industries (top 5 per industry)
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

windowSpec = Window.partitionBy("NAICS2_NAME").orderBy(col("SKILL_COUNT").desc())
top_skills_per_industry = industry_skills.withColumn("rank", row_number().over(windowSpec)) \
                                         .filter(col("rank") <= 5)

# Convert to Pandas for Plotly
pdf = pd.DataFrame(
    top_skills_per_industry.select("NAICS2_NAME", "SKILLS_NAME", "SKILL_COUNT").collect(),
    columns=["NAICS2_NAME", "SKILLS_NAME", "SKILL_COUNT"]
)

fig = px.bar(
    pdf,
    x="NAICS2_NAME",
    y="SKILL_COUNT",
    color="SKILLS_NAME",
    title="Top In-Demand Skills by Industry",
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title="Industry",
    yaxis_title="Skill Count",
    legend_title="Skill",
    barmode="stack",
    xaxis_tickangle=-45
)

HTML(to_html(fig, include_plotlyjs='cdn'))


<>:6: SyntaxWarning:

invalid escape sequence '\s'

<>:6: SyntaxWarning:

invalid escape sequence '\s'

/tmp/ipykernel_10391/2673313181.py:6: SyntaxWarning:

invalid escape sequence '\s'



Across most industries, the most in-demand skills include data analysis, communication, problem-solving, and management. These skills are highly valued because they are essential for making data-driven decisions, collaborating effectively in teams, and navigating complex business challenges—regardless of industry.


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [30]:
from pyspark.sql.functions import count, percentile_approx

# Group by ONET and LOT occupation names
onet_summary = df.groupBy("ONET_NAME", "LOT_OCCUPATION_NAME") \
    .agg(
        count("*").alias("JOB_COUNT"),
        percentile_approx("SALARY_FROM", 0.5).alias("MEDIAN_SALARY")
    )

# Convert to Pandas with all grouped columns
pdf = pd.DataFrame(
    onet_summary.collect(),
    columns=["ONET_NAME", "LOT_OCCUPATION_NAME", "JOB_COUNT", "MEDIAN_SALARY"]
)

# Create the bubble chart
fig = px.scatter(
    pdf,
    x="LOT_OCCUPATION_NAME",
    y="MEDIAN_SALARY",
    size="JOB_COUNT",
    color="MEDIAN_SALARY",
    title="Median Salary by ONET Occupation",
    color_continuous_scale="Blues",
    hover_name="ONET_NAME"  # Optional: show ONET_NAME on hover
)

#  Styling
fig.update_layout(
    font_family="Arial",
    title_font_size=18,
    xaxis_title="Occupation (LOT)",
    yaxis_title="Median Salary",
    xaxis_tickangle=-45
)

HTML(to_html(fig, include_plotlyjs='cdn'))




In the chart, Computer Systems Engineers have a higher median salary compared to Business Intelligence Analysts, reflecting stronger compensation in technical engineering roles. Meanwhile, Data Mining Specialists show the lowest median salary among the ONET occupations, suggesting either lower demand or entry-level positioning in the market.

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [31]:
from pyspark.sql.functions import count
import pandas as pd
import plotly.graph_objects as go

# Step 1: Count transitions (assuming each record is a job)
transitions_df = df.groupBy("SOC_2021_2_NAME", "SOC_2021_3_NAME") \
                   .agg(count("*").alias("TRANSITIONS"))

# Step 2: Convert to Pandas
pdf = pd.DataFrame(
    transitions_df.collect(),
    columns=["SOC_2021_2_NAME", "SOC_2021_3_NAME", "TRANSITIONS"]
)

# Step 3: Create unique list of node names (source + target)
labels = list(pd.unique(pdf[["SOC_2021_2_NAME", "SOC_2021_3_NAME"]].values.ravel()))

# Map each SOC name to an index for Sankey diagram
label_index = {label: i for i, label in enumerate(labels)}

# Map source/target names to indexes
pdf["source_idx"] = pdf["SOC_2021_2_NAME"].map(label_index)
pdf["target_idx"] = pdf["SOC_2021_3_NAME"].map(label_index)

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=20,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="lightblue"
    ),
    link=dict(
        source=pdf["source_idx"],
        target=pdf["target_idx"],
        value=pdf["TRANSITIONS"]
    )
)])

fig.update_layout(
    title_text="Job Transitions Between SOC Occupation Levels",
    font=dict(size=12, family="Arial")
)

HTML(to_html(fig, include_plotlyjs='cdn'))



The Sankey diagram shows a single transition from Computer and Mathematical Occupations to Mathematical Science Occupations, indicating that this is the only job flow captured in the current dataset. This may suggest that the data is focused on a specific occupational pathway—such as transitions within data or analytics-related roles—or that other occupation transitions were not recorded or available.